In [125]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [126]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(40000)
user_df = pd.read_csv('Users.csv')
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

/var/folders/tr/1ykjgs6x0dlg5g_7jm6k81qr0000gn/T/ipykernel_8100/1239598326.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('Books.csv')


In [127]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)

In [128]:
book_user_rating


,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0060973129,Decision in Normandy,Carlo D'Este,2954,8
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,208059,6
2,0399135782,The Kitchen God's Wife,Amy Tan,11676,9
3,0399135782,The Kitchen God's Wife,Amy Tan,148712,10
4,0887841740,The Middle Stories,Sheila Heti,92861,0
...,...,...,...,...,...
35915,0425086615,"\Dearest Amanda...\"": An Executives Advice to ...",Eliza Collins,259901,0
35916,0553234102,R Is for Rocket,Ray Bradbury,259901,0
35917,0679861548,Wolverine: Duty and Honor (X-Men Marvel Comics),Francine Hughes,259901,7
35918,0553217984,"In Serena's Web (Loveswept, No 189)",Kay Hooper,266657,0


In [129]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

In [130]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID', 
                                                          columns='unique_id_book', 
                                                          values='Book-Rating').fillna(0)

In [131]:
users_books_pivot_matrix_df.head()

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,26167,26168,26169,26170,26171,26172,26173,26174,26175,26176
User-ID,,,,,,,,,,,,,,,,,,,,,
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [133]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [134]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [135]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[-1.19364551e-34,  1.66181982e-35, -1.57584337e-18, ...,
        -4.39888283e-34,  0.00000000e+00,  0.00000000e+00],
       [-6.61779258e-34,  3.23349658e-35, -8.73544445e-18, ...,
         7.70263120e-35,  0.00000000e+00,  0.00000000e+00],
       [ 2.06526450e-34, -3.14547040e-35,  5.57922106e-18, ...,
         7.41836213e-34,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 2.24529917e-34, -2.99135965e-35,  5.30525805e-18, ...,
         7.99648623e-34,  0.00000000e+00,  0.00000000e+00],
       [-2.03456901e-35,  2.83206417e-36, -4.23963316e-19, ...,
        -7.87200009e-35,  0.00000000e+00,  0.00000000e+00],
       [-2.09394257e-34,  4.04999373e-35, -2.47940803e-18, ...,
        -8.11636542e-34,  0.00000000e+00,  0.00000000e+00]])

In [136]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])

In [107]:
# def top_cosine_similarity(book_user_rating,data, book_id, top_n=10):
#     index = book_user_rating.index[book_user_rating['ISBN'] == '1558746218'].tolist() 
#     book_row = data[index, :]
#     magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
#     similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
#     sort_indexes = np.argsort(-similarity)
#     sort_indexes = sort_indexes[0]
#     return sort_indexes[:top_n]

# def similar_books(book_user_rating, book_id, top_indexes):
#     print(top_indexes.shape)
#     print('Recommendations for {0}: \n'.format(
#     book_user_rating[book_user_rating.ISBN == book_id]['Book-Title'].values[0]))
#     for id in top_indexes + 1:
#         print(book_user_rating.iloc[id]['Book-Title'])
  

In [151]:
k = 50
movie_id =205  
top_n = 5
sliced = Vt.T[:, :k] # representative data

similar_books(book_user_rating, 205, top_cosine_similarity(sliced, movie_id, top_n))

Recommendations for The Firm: 

Murder at the Kennedy Center (Capital Crime Mysteries)
Small Miracles of Love &amp; Friendship: Remarkable Coincidences of Warmth and Devotion
Tapping the Dream Tree
A Rage To Kill and Other True Cases : Anne Rule's Crime Files, Vol. 6 (Ann Rule's Crime Files)
The Vampire Princess


/var/folders/tr/1ykjgs6x0dlg5g_7jm6k81qr0000gn/T/ipykernel_8100/437162856.py:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)


In [124]:
# k = 50
# movie_id =1893699501  
# top_n = 5
# sliced = Vt.T[:, :k] # representative data

# similar_books(book_user_rating, '0671704664', top_cosine_similarity(book_user_rating,sliced, movie_id, top_n))

(5,)
Recommendations for And Then There Were None: 

The Kitchen God's Wife
Dead Famous
Garzanti - Gli Elefanti: Paula
Gita a Tindari (La memoria)
Rendez-vous a Bagdad


/var/folders/tr/1ykjgs6x0dlg5g_7jm6k81qr0000gn/T/ipykernel_8100/3654495838.py:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)


In [58]:
book_user_rating[book_user_rating['ISBN'] == '1558746218']

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
3,1558746218,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,256233,0


In [110]:
book_user_rating.index[book_user_rating['ISBN'] == '9781780221359'].tolist()

[]

In [71]:
book_id = '1893699501'
print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.ISBN == book_id]['Book-Title'].values[0]))

Recommendations for Missouri Madhouse (American Chillers): 



In [72]:
book_user_rating[book_user_rating.ISBN == book_id]['Book-Title']

35837    Missouri Madhouse (American Chillers)
Name: Book-Title, dtype: object

In [104]:
print(book_user_rating.iloc[1]['Book-Title'])

The Kitchen God's Wife


In [148]:
index = book_user_rating.index[book_user_rating['Book-Author'] == 'John Grisham'].tolist()
print(index)

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 527, 528, 529, 530, 531, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 791, 792, 793, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1211, 1212, 1213, 1214, 1714, 1715, 1716, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919, 2920, 2921, 2922, 2923, 2924, 2925, 2926, 2927, 2928, 2966, 2967, 2968, 2969, 2970, 2971, 2972, 2973, 2974, 2975, 2976, 2977, 2978, 2979, 3490, 3692, 3693, 3694, 3695, 3696, 3697, 3698, 3699, 3700, 3701, 3702, 3703, 3704, 3705, 3706, 3707, 3821, 3822, 3823, 3824, 4921, 4922, 4923, 4924, 4925, 4926, 4927, 4928, 4929, 5393, 5886, 6359, 6360, 6370, 6371, 6372, 6373, 6374, 6375, 6376, 6377, 6465, 6466, 6467, 6468, 6469, 6470, 6471, 6472, 6473, 6474, 6475, 6476, 6477,

In [118]:
book_user_rating[book_user_rating.index.isin(index)]

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating


In [150]:
book_user_rating.iloc[918]

ISBN                044021145X
Book-Title            The Firm
Book-Author       John Grisham
User-ID                 225087
Book-Rating                  0
unique_id_book             205
Name: 918, dtype: object

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating,unique_id_book
